In [3]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler


chat = ChatOpenAI(
  temperature=0.1, 
  streaming=True,
  callbacks=[
    StreamingStdOutCallbackHandler()
  ]
)

In [4]:
#FewShot- 모델에게 더 나은 대답을 할 수 있도록 하는 예제들을 준다
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.prompts import PromptTemplate

examples = [
  {
    "question":"What do you know about France?",
    "answer": """Here is what I know: 
    Capital:paris
    Language:French
    Food:Wine and Cheese
    Currency:Euro
    """,
  },
  {
    "question":"What do you know about Italy?",
    "answer": """Here is what I know: 
    Capital:Rome
    Language:Italian
    Food:Pizza and Pasta
    Currency:Euro
    """,
  },
    {
    "question":"What do you know about Greece?",
    "answer": """Here is what I know: 
    Capital:Athene
    Language:Greek
    Food:Souvlaki adn Feta Cheese
    Currency:Euro
    """,
  }
]

example_template = """
  Human:{question}
  AI:{answer}
"""

example_prompt = PromptTemplate.from_template(example_template)

prompt = FewShotPromptTemplate(
  example_prompt=example_prompt,
  examples=examples,
  #형식화된 모든 예제 마지막에 나오는 내용
  suffix="Human:What do you know about {country}",
  input_variables=["country"]
)

#prompt.format(country="Germany")
chain = prompt | chat

chain.invoke({
  "country":"Germany"
})

AI: Here is what I know:
    Capital: Berlin
    Language: German
    Food: Bratwurst and Sauerkraut
    Currency: Euro

AIMessageChunk(content='AI: Here is what I know:\n    Capital: Berlin\n    Language: German\n    Food: Bratwurst and Sauerkraut\n    Currency: Euro')

In [5]:
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.prompts import ChatPromptTemplate

examples = [
  {
    "country":"France",
    "answer": """Here is what I know: 
    Capital:paris
    Language:French
    Food:Wine and Cheese
    Currency:Euro
    """,
  },
  {
    "country":"Italy",
    "answer": """Here is what I know: 
    Capital:Rome
    Language:Italian
    Food:Pizza and Pasta
    Currency:Euro
    """,
  },
    {
    "country":"Greece",
    "answer": """Here is what I know: 
    Capital:Athene
    Language:Greek
    Food:Souvlaki adn Feta Cheese
    Currency:Euro
    """,
  }
]

example_prompt = ChatPromptTemplate.from_messages([
  ("human", "What do you know about {country}?"),
  ("ai", "{answer}")
])

example_prompt = FewShotChatMessagePromptTemplate(
  example_prompt=example_prompt,
  examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages([
  ("system","You are a geography expert, you give short answers"),
  example_prompt,
  ("human","What do you know about {country}")
])

#prompt.format(country="Germany")
chain = final_prompt | chat

chain.invoke({
  "country":"Germany"
})

Here is what I know: 
    Capital:Berlin
    Language:German
    Food:Bratwurst and Sauerkraut
    Currency:Euro

AIMessageChunk(content='Here is what I know: \n    Capital:Berlin\n    Language:German\n    Food:Bratwurst and Sauerkraut\n    Currency:Euro')

In [6]:
#예제들을 형식화하고, 예제의 양을 어마나 되는지 확인이 가능하다.
#설정해놓은 세팅값에 따라 prompt에 알맞은 예제를 골라준다.
from langchain.prompts.example_selector import LengthBasedExampleSelector

examples = [
  {
    "question":"What do you know about France?",
    "answer": """Here is what I know: 
    Capital:paris
    Language:French
    Food:Wine and Cheese
    Currency:Euro
    """,
  },
  {
    "question":"What do you know about Italy?",
    "answer": """Here is what I know: 
    Capital:Rome
    Language:Italian
    Food:Pizza and Pasta
    Currency:Euro
    """,
  },
    {
    "question":"What do you know about Greece?",
    "answer": """Here is what I know: 
    Capital:Athene
    Language:Greek
    Food:Souvlaki adn Feta Cheese
    Currency:Euro
    """,
  }
]

example_prompt = PromptTemplate.from_template("Human:{question}\nAI:{answer}")

example_selector = LengthBasedExampleSelector(
  examples=examples,
  #형식화 양이 얼마나 되는지 알기 위해
  example_prompt=example_prompt,
  #예제 양
  max_length=180
)

prompt = FewShotPromptTemplate(
  example_prompt=example_prompt,
  #예제의 양이 얼마나 되던 example prompt를 사용하여 모든 example들을 형식화
  #examples=examples,
  #max_length에 따라 예제의 양을 정해준다.
  example_selector=example_selector,
  #형식화된 모든 예제 마지막에 나오는 내용
  suffix="Human:What do you know about {country}?",
  input_variables=["country"]
)

prompt.format(country="Brazil")


'Human:What do you know about France?\nAI:Here is what I know: \n    Capital:paris\n    Language:French\n    Food:Wine and Cheese\n    Currency:Euro\n    \n\nHuman:What do you know about Italy?\nAI:Here is what I know: \n    Capital:Rome\n    Language:Italian\n    Food:Pizza and Pasta\n    Currency:Euro\n    \n\nHuman:What do you know about Greece?\nAI:Here is what I know: \n    Capital:Athene\n    Language:Greek\n    Food:Souvlaki adn Feta Cheese\n    Currency:Euro\n    \n\nHuman:What do you know about Brazil?'

In [7]:
from langchain.prompts.example_selector.base import BaseExampleSelector
  
examples = [
  {
    "question":"What do you know about France?",
    "answer": """Here is what I know: 
    Capital:paris
    Language:French
    Food:Wine and Cheese
    Currency:Euro
    """,
  },
  {
    "question":"What do you know about Italy?",
    "answer": """Here is what I know: 
    Capital:Rome
    Language:Italian
    Food:Pizza and Pasta
    Currency:Euro
    """,
  },
    {
    "question":"What do you know about Greece?",
    "answer": """Here is what I know: 
    Capital:Athene
    Language:Greek
    Food:Souvlaki adn Feta Cheese
    Currency:Euro
    """,
  }
]

class RandomExampleSelector(BaseExampleSelector):

  def __init__(self, examples):
      self.examples = examples

  def add_example(self, example):
    self.append(example)

  def select_examples(self, input_variables):
    from random import choice
    return [choice(self.examples)]
  
example_prompt = PromptTemplate.from_template("Human:{question}\nAI:{answer}")

example_selector = RandomExampleSelector(
  examples=examples,
)

prompt = FewShotPromptTemplate(
  example_prompt=example_prompt,
  #예제의 양이 얼마나 되던 example prompt를 사용하여 모든 example들을 형식화
  #examples=examples,
  #max_length에 따라 예제의 양을 정해준다.
  example_selector=example_selector,
  #형식화된 모든 예제 마지막에 나오는 내용
  suffix="Human:What do you know about {country}?",
  input_variables=["country"]
)

prompt.format(country="Brazil")


'Human:What do you know about Italy?\nAI:Here is what I know: \n    Capital:Rome\n    Language:Italian\n    Food:Pizza and Pasta\n    Currency:Euro\n    \n\nHuman:What do you know about Brazil?'

In [8]:
from  langchain.prompts import load_prompt

#prompt = load_prompt("./prompt.json")
prompt = load_prompt("./prompt.yaml")

prompt.format(country="Germany")


'What is the capital of Germany?'

In [9]:
#prompt의 메모리들을 모으는
#많은 prompt들을 하나로 합칠 수 있도록 해준다.
from langchain.prompts.pipeline import PipelinePromptTemplate

intro = PromptTemplate.from_template(
  """
  You are a role playing assistant.
  And you are impersonatinga {character}
  """
)

example = PromptTemplate.from_template(
  """
  This is an example of how you talk:

  Human: {example_question}
  You: {example_answer}
"""
)

start = PromptTemplate.from_template(
  """
  Start now!

  Human: {question}
  you:
"""
)

final = PromptTemplate.from_template(
  """
  {intro}

  {example}

  {start}
"""
)

prompts = [
  ("intro", intro),
  ("example", example),
  ("start", start)
]

full_prompt = PipelinePromptTemplate(
  final_prompt=final,
  pipeline_prompts=prompts
)

full_prompt.format(
  character="Pirate",
  example_question="What is your location?",
  example_answer="Arrrg! That is a secret!! Arg arg!!",
  question="What is your fav food?"
)

'\n  \n  You are a role playing assistant.\n  And you are impersonatinga Pirate\n  \n\n  \n  This is an example of how you talk:\n\n  Human: What is your location?\n  You: Arrrg! That is a secret!! Arg arg!!\n\n\n  \n  Start now!\n\n  Human: What is your fav food?\n  you:\n\n'

In [10]:
chain = full_prompt | chat

chain.invoke({
  "character":"Pirate",
  "example_question":"What is your location?",
  "example_answer":"Arrrg! That is a secret!! Arg arg!!",
  "question":"What is your fav food?"
})


Arrr matey! Me favorite grub be a hearty plate of salted beef and hardtack! Aye, nothing beats a good meal after a long day of pillaging and plundering on the high seas! Arrr!

AIMessageChunk(content='Arrr matey! Me favorite grub be a hearty plate of salted beef and hardtack! Aye, nothing beats a good meal after a long day of pillaging and plundering on the high seas! Arrr!')

In [11]:
#LLM의 응답을 저장할 수 있다.
# 같은 질문을 계속 하고 응답이 같다면 계속 답변을 만들지 않고 이미 답변한 답을 캐싱을 이용하여 저장하여 재사용할 수 있다.
from langchain.globals import set_llm_cache, set_debug
#SQLiteCache-데이터베이스에 캐싱할 수 있도록 해준다.
from langchain.cache import InMemoryCache, SQLiteCache

#response가 메모리에 저장됨
set_llm_cache(InMemoryCache())
#set_llm_cache(SQLiteCache("cache.db"))
#무슨일을 하고있는지에 대한 로그를 보여준다.
set_debug(True)

chat.predict("How do you make italian pasta?")

[llm/start] [1:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: How do you make italian pasta?"
  ]
}
To make Italian pasta, you will need the following ingredients:

- 2 cups of all-purpose flour
- 2 large eggs
- Pinch of salt

Here is a step-by-step guide to making Italian pasta:

1. On a clean work surface, pour the flour and create a well in the center.
2. Crack the eggs into the well and add a pinch of salt.
3. Using a fork, gradually mix the eggs into the flour until a dough forms.
4. Knead the dough for about 10 minutes until it is smooth and elastic.
5. Wrap the dough in plastic wrap and let it rest for at least 30 minutes.
6. After resting, roll out the dough using a pasta machine or a rolling pin until it is thin.
7. Cut the dough into desired shapes, such as fettuccine or spaghetti.
8. Cook the pasta in a large pot of boiling salted water for 2-3 minutes or until al dente.
9. Drain the pasta and toss with your favorite sauce or toppings.

Enjoy your 

'To make Italian pasta, you will need the following ingredients:\n\n- 2 cups of all-purpose flour\n- 2 large eggs\n- Pinch of salt\n\nHere is a step-by-step guide to making Italian pasta:\n\n1. On a clean work surface, pour the flour and create a well in the center.\n2. Crack the eggs into the well and add a pinch of salt.\n3. Using a fork, gradually mix the eggs into the flour until a dough forms.\n4. Knead the dough for about 10 minutes until it is smooth and elastic.\n5. Wrap the dough in plastic wrap and let it rest for at least 30 minutes.\n6. After resting, roll out the dough using a pasta machine or a rolling pin until it is thin.\n7. Cut the dough into desired shapes, such as fettuccine or spaghetti.\n8. Cook the pasta in a large pot of boiling salted water for 2-3 minutes or until al dente.\n9. Drain the pasta and toss with your favorite sauce or toppings.\n\nEnjoy your homemade Italian pasta!'

In [12]:
from langchain.callbacks import get_openai_callback

# with 코드 블록 안에서 모델을 호출하고, print 에 usage를 넣어 사용
with get_openai_callback() as usage:
  a = chat.predict("What is the recipe for cake")
  b = chat.predict("What is the recipe for bread")
  print(a, b, "\n")
  #.... 중간에 들어가는 코드들은 모두 usage에들어간다
  #사용량 출력- Prompt 혹은 완료에 얼마나 많은 토근을 사용했는지, 비용을 알려줌
  print(usage)
  #print(usage.total_cost)
  #print(usage.total_tokens)

[llm/start] [1:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: What is the recipe for cake"
  ]
}
Here is a basic recipe for a simple vanilla cake:

Ingredients:
- 1 1/2 cups all-purpose flour
- 1 1/2 teaspoons baking powder
- 1/2 teaspoon salt
- 1/2 cup unsalted butter, softened
- 1 cup granulated sugar
- 2 large eggs
- 1 teaspoon vanilla extract
- 1/2 cup milk

Instructions:
1. Preheat your oven to 350°F (180°C) and grease and flour a 9-inch round cake pan.
2. In a medium bowl, whisk together the flour, baking powder, and salt.
3. In a separate large bowl, cream together the butter and sugar until light and fluffy.
4. Beat in the eggs one at a time, then stir in the vanilla extract.
5. Gradually mix in the dry ingredients, alternating with the milk, until just combined.
6. Pour the batter into the prepared cake pan and smooth the top with a spatula.
7. Bake in the preheated oven for 25-30 minutes, or until a toothpick inserted into the center comes out clean